In [1]:
!pip uninstall opencv-python -y
!pip install opencv-contrib-python

import cv2
import numpy as np
from scipy.interpolate import UnivariateSpline

def detect_and_match_features_orb(img1, img2, orb):
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    if des1 is None or des2 is None:
        return np.array([]), np.array([])
    
    # Use BFMatcher for ORB (binary descriptors)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    
    if len(matches) < 4:
        return np.array([]), np.array([])
    
    # Take best matches
    good_matches = matches[:50]  # Top 50 matches
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches])
    return pts1, pts2

def estimate_homography(pts1, pts2):
    if len(pts1) >= 4 and len(pts2) >= 4:
        H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)
        if H is None:
            return np.eye(3)
        return H
    else:
        return np.eye(3)

def smooth_trajectory(trajectory, smoothing_factor=5):
    trajectory = np.array(trajectory)
    smoothed = []
    for i in range(trajectory.shape[1]):
        x = np.arange(trajectory.shape[0])
        y = trajectory[:, i]
        spl = UnivariateSpline(x, y, s=smoothing_factor)
        smoothed.append(spl(x))
    return np.array(smoothed).T

def streaming_video_stabilization_orb(input_path, output_path):
    """
    Fast streaming video stabilization using ORB features
    """
    # Open input video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video")
        return
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video info: {total_frames} frames, {w}x{h}, {fps:.2f} FPS")
    
    # Setup output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))
    
    # Initialize ORB detector (much faster than SIFT)
    orb = cv2.ORB_create(nfeatures=1000)
    
    # Read first frame
    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Could not read first frame")
        cap.release()
        out.release()
        return
    
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    # Initialize lists to store trajectory and transforms
    transforms = []
    trajectory = [np.eye(3)]
    
    frame_count = 1
    print("Phase 1: Computing transforms (using fast ORB)...")
    
    # Phase 1: Compute all transforms
    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        pts1, pts2 = detect_and_match_features_orb(prev_gray, curr_gray, orb)
        H = estimate_homography(pts1, pts2)
        transforms.append(H)
        
        # Build cumulative trajectory
        trajectory.append(trajectory[-1] @ H)
        
        prev_gray = curr_gray
        frame_count += 1
        
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")
    
    print("Phase 2: Smoothing trajectory...")
    
    # Extract translation and rotation parameters
    params = []
    for H in trajectory:
        dx = H[0, 2]
        dy = H[1, 2]
        da = np.arctan2(H[1, 0], H[0, 0])
        params.append([dx, dy, da])
    
    # Smooth trajectory
    smoothed_params = smooth_trajectory(params, smoothing_factor=10)
    
    # Calculate smoothed transforms
    new_transforms = []
    for i in range(len(smoothed_params)):
        dx, dy, da = smoothed_params[i]
        cos_a = np.cos(da)
        sin_a = np.sin(da)
        new_H = np.array([[cos_a, -sin_a, dx],
                          [sin_a,  cos_a, dy],
                          [0,      0,     1]], dtype=np.float32)
        new_transforms.append(new_H)
    
    print("Phase 3: Applying stabilization and writing video...")
    
    # Reset to beginning
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Apply stabilization transform
        if frame_idx < len(new_transforms):
            H = new_transforms[frame_idx]
            
            # Validate transform matrix
            if H is not None and not np.isnan(H).any() and not np.isinf(H).any():
                try:
                    frame_stabilized = cv2.warpPerspective(frame, H, (w, h), 
                                                         borderMode=cv2.BORDER_REFLECT)
                except:
                    frame_stabilized = frame
            else:
                frame_stabilized = frame
        else:
            frame_stabilized = frame
        
        # Write stabilized frame directly to output video
        out.write(frame_stabilized)
        
        frame_idx += 1
        if frame_idx % 100 == 0:
            print(f"Stabilized and wrote {frame_idx}/{total_frames} frames")
    
    # Cleanup
    cap.release()
    out.release()
    print(f"Stabilized video saved at: {output_path}")


In [3]:
input_path = "DJI_20250411113208_0019_D.MP4"
output_path = "BlineORB_stabilized_output.mp4"

streaming_video_stabilization_orb(input_path, output_path)

Video info: 7170 frames, 3840x2160, 29.97 FPS
Phase 1: Computing transforms (using fast ORB)...
Processed 100/7170 frames
Processed 200/7170 frames
Processed 300/7170 frames
Processed 400/7170 frames
Processed 500/7170 frames
Processed 600/7170 frames
Processed 700/7170 frames
Processed 800/7170 frames
Processed 900/7170 frames
Processed 1000/7170 frames
Processed 1100/7170 frames
Processed 1200/7170 frames
Processed 1300/7170 frames
Processed 1400/7170 frames
Processed 1500/7170 frames
Processed 1600/7170 frames
Processed 1700/7170 frames
Processed 1800/7170 frames
Processed 1900/7170 frames
Processed 2000/7170 frames
Processed 2100/7170 frames
Processed 2200/7170 frames
Processed 2300/7170 frames
Processed 2400/7170 frames
Processed 2500/7170 frames
Processed 2600/7170 frames
Processed 2700/7170 frames
Processed 2800/7170 frames
Processed 2900/7170 frames
Processed 3000/7170 frames
Processed 3100/7170 frames
Processed 3200/7170 frames
Processed 3300/7170 frames
Processed 3400/7170 fr